## Mount google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import necessary libraries

In [0]:
import pandas as pd
import urllib.request as request
import progressbar
import time
import threading
import numpy as np
import os
from PIL import Image

## Set directory path to save downloaded images

In [0]:
train_file = "/content/drive/Shared drives/AISight/Google Dataset/Train_GCC-training.tsv"
photos_dir = "/content/drive/Shared drives/AISight/Google Dataset/gadgets"

## Read csv

In [0]:
df = pd.read_csv(train_file, delimiter="\t", names=['Captions', 'Urls'])

## Download Google Conceptual dataset
Try to download the image from url, if no response from the url for 8 seconds then it will skip that image. This saves a lot of time as many urls are not working.

In [0]:
def download_images(start, end):
  print (start, end)
  error = []
  bar = progressbar.ProgressBar(max_value=(end-start))
  for i in range(start, end):
    # print (i)
    try:
      request.urlopen(links[i], timeout = 8)
      request.urlretrieve(links[i],  photos_dir + "/" + str(i) + ".jpg")
    except:
      error.append(i)
    bar.update(i-start)
  return error

In [0]:
def thread_fun(start, end):
  error = download_images(start, end)
  with open(os.path.join(error_file, "error_" + str(start) + "_" + str(end) + ".txt"), 'w+') as file:
    for e in error:
      file.write(str(e) + ",") 

In [0]:
for i in range(0,60):
  thread_fun(i*1000,i*1000+1000)

In [0]:
print(len([name for name in os.listdir(photos_dir)]))

In [0]:
print(len(error) , "not downloaded")

## Check and remove file if it is not an image
Try to open the file using PIL library's Image module and if fails delete it - **Multithreading** 

In [0]:
num_thread = 30
photos_path = [path for path in os.listdir(photos_dir)]
per_thread_photos = int(len(photos_path)/num_thread)

In [0]:
def check_photos(start,end):
  bar = progressbar.ProgressBar(max_value=(end-start))
  for i, path in zip(range(start,end), photos_path[start:end]):
    try:
      Image.open(os.path.join(photos_dir, path))
    except:
      os.remove(os.path.join(photos_dir, path))
    bar.update(i-start)

In [0]:
t = []
for i in range(num_thread):
    arg1 = i * per_thread_photos
    if i == (num_thread-1):
        arg2 = len(photos_path)
    else:
        arg2 = i * per_thread_photos + per_thread_photos
    # print(arg1, arg2)
    t.append(threading.Thread(target=check_photos, args=(arg1, arg2, )))
    t[i].start()
      
for i in range(num_thread):
    t[i].join()